In [ ]:
! pip install faiss-cpu==1.7.4 mistralai

# Basic RAG

Retrieval-augmented generation (RAG) is an AI framework that synergizes the capabilities of LLMs and information retrieval systems. It's useful to answer questions or generate content leveraging external knowledge.
---
 There are two main steps in RAG :

1.  retrieval: retrieve relevant information from a knowledge base with text embeddings stored in a vector store
2.  generation: insert the relevant information to the prompt for the LLM to generate information.

In this guide, we will walk through a very basic example of RAG with five implementations:


* RAG from scratch with Mistral
* RAG with Mistral and LangChain
* RAG with Mistral and LlamaIndex
* RAG with Mistral and Haystack
* RAG with Mistral and Vercel AI SDK




---



# RAG from scratch

### Import needed packages
The first step is to install the needed packages `mistralai` and `faiss-cpu` and import them:





In [ ]:
import os
import numpy as np
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
api_key = "lmQqjB3jLr2lP0OjFWsKuuzFBssI0U6x"
client = MistralClient(api_key=api_key)

# Getting Data




In [ ]:
import requests

response = requests.get(
    "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
)
text = response.text
# print(text)

#Save as Text file

In [ ]:
file_name = "essay.txt"
with open(file_name, 'w') as file:
    file.write(text)

#Split document into chunks

In a RAG system,
- it is crucial to split the document into smaller chunks so that it's more effective to identify and retrieve the most relevant information in the retrieval process later.
- In this example, we simply split our text by character, combine 2048 characters into each chunk, and we get 37 chunks.


In [ ]:
chunk_size = 2048
chunks = [text[i:i+ chunk_size] for i in range( 0 , len(text) , chunk_size)]
len(chunks)
# print(chunks)

37

## Considerations:

- **Chunk size:** Depending on your specific use case, it may be necessary to customize or experiment with different chunk sizes and chunk overlap to achieve optimal performance in RAG.

 `For example`, **smaller chunks can be more beneficial in retrieval processes** , as larger text chunks often contain filler text that can obscure the semantic representation. As such, using smaller text chunks in the retrieval process can enable the RAG system to identify and extract relevant information more effectively and accurately. However, it's worth considering the trade-offs that come with using smaller chunks, such as increasing processing time and computational resources.

- **How to split:** While the simplest method is to split the text by character, there are other options depending on the use case and document structure.

  `For example`, to avoid exceeding token limits in API calls, it may be necessary to split the text by tokens. To maintain the cohesiveness of the chunks, it can be useful to split the text by sentences, paragraphs, or HTML headers. If working with code, it's often recommended to split by meaningful code chunks for example using an Abstract Syntax Tree (AST) parser.


#Get embeddings for each text chunk

For each text chunk, we then need to create text embeddings, which are numeric representations of the text in the vector space. Words with similar meanings are expected to be in closer proximity or have a shorter distance in the vector space.

To create an embedding, use Mistral AI's embeddings API endpoint and the embedding model `mistral-embed`. We create a `get_text_embedding` to get the embedding from a single text chunk and then we use list comprehension to get text embeddings for all text chunks.

In [ ]:
def get_text_embedding(data):
  embeddings_batch_response = client.embeddings(
      model = 'mistral-embed',
      input = data
  )
  return embeddings_batch_response.data[0].embedding

In [ ]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [ ]:
text_embeddings

array([[-3.95202637e-02,  7.75756836e-02, -8.82148743e-05, ...,
        -1.26342773e-02, -2.12402344e-02, -2.50816345e-03],
       [-3.19213867e-02,  7.21435547e-02,  2.99835205e-02, ...,
        -1.08413696e-02, -1.19628906e-02, -7.66372681e-03],
       [-5.89599609e-02,  6.12487793e-02,  1.26419067e-02, ...,
        -2.25372314e-02,  4.67681885e-03, -6.26754761e-03],
       ...,
       [-5.52978516e-02,  6.89697266e-02,  2.69622803e-02, ...,
        -2.54211426e-02, -2.52227783e-02, -2.68859863e-02],
       [-3.90014648e-02,  5.63049316e-02,  4.76684570e-02, ...,
        -1.77001953e-02,  9.33074951e-03, -8.72039795e-03],
       [-2.99377441e-02,  5.81054688e-02,  1.70898438e-02, ...,
        -1.61132812e-02, -1.79290771e-02, -4.35791016e-02]])

In [ ]:
len(text_embeddings[0])

1024

# Load into a vector database

Once we get the text embeddings, a common practice is to store them in a vector database for efficient processing and retrieval. There are several vector database to choose from.

In our simple example, we are using an open-source vector database `Faiss`, which allows for efficient similarity search.

With Faiss, we instantiate an instance of the Index class, which defines the indexing structure of the vector database. We then add the text embeddings to this indexing structure.

In [ ]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

# Considerations:

- **Vector database:** When selecting a vector database, there are several factors to consider including speed, scalability, cloud management, advanced filtering, and open-source vs. closed-source.

# Create embeddings for a question

Whenever users ask a question, we also need to create embeddings for this question using the same embedding models as before.

In [ ]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_text_embedding(question)])

In [ ]:
question_embeddings

array([[-0.05456543,  0.03518677,  0.03723145, ..., -0.02763367,
        -0.00327873,  0.00323677]])

# Considerations:

- **Hypothetical Document Embeddings (HyDE):** In some cases, the user's question might not be the most relevant query to use for identifying the relevant context.

  Instead, it maybe more effective to generate a hypothetical answer or a hypothetical document based on the user's query and use the embeddings of the generated text to retrieve similar text chunks.

# Retrieve similar chunks from the vector database

We can perform a search on the vector database with `index.search`, which takes two arguments: the first is the vector of the question embeddings, and the second is the number of similar vectors to retrieve. This function returns the distances and the indices of the most similar vectors to the question vector in the vector database.

Then based on the returned indices, we can retrieve the actual relevant text chunks that correspond to those indices.

In [ ]:
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [ ]:
print(D, "\n", I, "\n", retrieved_chunk)

[[0.41638768 0.46976998]] 
 [[0 3]] 
 ['\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had 

# Considerations:

- **Retrieval methods:** There are a lot different retrieval strategies.

 In our example, we are showing a simple similarity search with embeddings. Sometimes when there is metadata available for the data, it's better to filter the data based on the metadata first before performing similarity search. There are also other statistical retrieval methods like TF-IDF and BM25 that use frequency and distribution of terms in the document to identify relevant text chunks.

- **Retrieved document:** Do we always retrieve individual text chunk as it is? Not always.
Sometime, we would like to include more context around the actual retrieved text chunk. We call the actual retrieved text chunk "child chunk" and our goal is to retrieve a larger "parent chunk" that the "child chunk" belongs to.

  On occasion, we might also want to provide weights to our retrieve documents.
  
   `For example`, a time-weighted approach would help us retrieve the most recent document.
One common issue in the retrieval process is the "lost in the middle" problem where the information in the middle of a long context gets lost. Our models have tried to mitigate this issue.

  `For example`, in the passkey task, our models have demonstrated the ability to find a "needle in a haystack" by retrieving a randomly inserted passkey within a long prompt, up to 32k context length. However, it is worth considering experimenting with reordering the document to determine if placing the most relevant chunks at the beginning and end leads to improved results.


# Combine context and question in a prompt and generate response

Finally, we can offer the retrieved text chunks as the context information within the prompt. Here is a prompt template where we can include both the retrieved text and user question in the prompt.



In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

Then we can use the Mistral chat completion API to chat with a Mistral model (e.g., mistral-medium-latest) and generate answers based on the user question and the context of the question

In [ ]:
def run_mistral(user_message, model="mistral-medium-latest"):
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)

"The two main things the author worked on before college, outside of school, were writing and programming. In terms of writing, they wrote short stories, which they described as having hardly any plot and just characters with strong feelings. For programming, they wrote programs in an early version of Fortran on an IBM 1401 computer, which they had access to through their school district. The computer was located in the basement of their junior high school, and they used punch cards to input their programs and print the results. However, the author notes that they couldn't figure out what to do with the computer and that their programs likely didn't do much, as they didn't have any data stored on punched cards. They do remember the moment they learned that programs could fail to terminate on a machine without time-sharing."

#### Considerations:
- Prompting techniques: Most of the prompting techniques can be used in developing a RAG system as well. For example, we can use few-shot learning to guide the model’s answers by providing a few examples. Additionally, we can explicitly instruct the model to format answers in a certain way.


In the next sections, we are going to show you how to do a similar basic RAG with some of the popular RAG frameworks. We will start with LlamaIndex and add other frameworks in the future.


# LangChain

In [20]:
!pip install langchain langchain-mistralai langchain_community

SyntaxError: invalid syntax (<ipython-input-21-5e3c6796fd57>, line 1)